In [4]:
import pandas as pd
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

In [2]:
df=pd.read_csv('Truth_Seeker_Model_Dataset.csv')

In [3]:
df.head(3)

,Unnamed: 0,author,statement,target,BinaryNumTarget,manual_keywords,tweet,5_label_majority_answer,3_label_majority_answer
0,0,D.L. Davis,End of eviction moratorium means millions of A...,True,1.0,"Americans, eviction moratorium",@POTUS Biden Blunders - 6 Month Update\n\nInfl...,Mostly Agree,Agree
1,1,D.L. Davis,End of eviction moratorium means millions of A...,True,1.0,"Americans, eviction moratorium",@S0SickRick @Stairmaster_ @6d6f636869 Not as m...,NO MAJORITY,Agree
2,2,D.L. Davis,End of eviction moratorium means millions of A...,True,1.0,"Americans, eviction moratorium",THE SUPREME COURT is siding with super rich pr...,Agree,Agree


In [5]:
model_name = "j-hartmann/emotion-english-roberta-large"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)


C:\Users\maddy\AppData\Local\Programs\Python\Python311\Lib\site-packages\huggingface_hub\file_download.py:133: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\maddy\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


In [9]:
text = df["statement"].tolist()

In [12]:
def classify_emotions(text, batch_size=8):
    # Tokenize the statements in batches
    predicted_emotions = []
    for i in range(0, len(text), batch_size):
        batch_statements = text[i:i + batch_size]
        inputs = tokenizer(batch_statements, padding=True, truncation=True, return_tensors="pt")

        with torch.no_grad():
            outputs = model(**inputs)

        predicted_labels = torch.argmax(outputs.logits, dim=1)
        label_to_emotion = {0: "anger", 1: "disgust", 2: "fear", 3: "joy", 4: "neutral", 5: "sadness", 6: "surprise"}
        batch_emotions = [label_to_emotion[label.item()] for label in predicted_labels]
        predicted_emotions.extend(batch_emotions)

    return predicted_emotions


In [13]:
sentiment = classify_emotions(text)

In [17]:
# Add the predicted emotions to the DataFrame
df["anger"] = [1 if emotion == "anger" else 0 for emotion in sentiment]
df["disgust"] = [1 if emotion == "disgust" else 0 for emotion in sentiment]
df["fear"] = [1 if emotion == "fear" else 0 for emotion in sentiment]
df["joy"] = [1 if emotion == "joy" else 0 for emotion in sentiment]
df["neutral"] = [1 if emotion == "neutral" else 0 for emotion in sentiment]
df["sadness"] = [1 if emotion == "sadness" else 0 for emotion in sentiment]
df["surprise"] = [1 if emotion == "surprise" else 0 for emotion in sentiment]


In [18]:
df.head(3)

,Unnamed: 0,author,statement,target,BinaryNumTarget,manual_keywords,tweet,5_label_majority_answer,3_label_majority_answer,anger,disgust,fear,joy,neutral,sadness,surprise
0,0,D.L. Davis,End of eviction moratorium means millions of A...,True,1.0,"Americans, eviction moratorium",@POTUS Biden Blunders - 6 Month Update\n\nInfl...,Mostly Agree,Agree,0,0,0,0,0,1,0
1,1,D.L. Davis,End of eviction moratorium means millions of A...,True,1.0,"Americans, eviction moratorium",@S0SickRick @Stairmaster_ @6d6f636869 Not as m...,NO MAJORITY,Agree,0,0,0,0,0,1,0
2,2,D.L. Davis,End of eviction moratorium means millions of A...,True,1.0,"Americans, eviction moratorium",THE SUPREME COURT is siding with super rich pr...,Agree,Agree,0,0,0,0,0,1,0


In [19]:
df.to_csv('truthseeker_emotions.csv', index=False)